# DB Systel MLOps Workshop

## Zusammenfassung der Befehle des Workshops

### 1 Einrichtung der Sandbox

Folgt der Anleitung in `readme.md` zum erstellen der Sandbox Umgebung.

### 2 Konfiguration unserer MLOps Sandbox

#### 2.1 Installiert Flyte in eurem k3d Cluster

Fügt das `flyteorg` helm repository hinzu. 

(erfordert die Installation des Kubernetes Package Managers `Helm`, führt dazu `brew install helm` oder siehe die Installationsseite, [hier](https://helm.sh/docs/intro/install/))


In [ ]:
!helm repo add flyteorg https://helm.flyte.org

Installiert die flyte dependencies, `flyte-deps`

In [ ]:
!helm install flyte-deps flyteorg/flyte-deps -n flyte --create-namespace -f https://raw.githubusercontent.com/flyteorg/flyte/master/charts/flyte-deps/values-sandbox.yaml

Hier haben wir einen `Minio Bucket` (ähnlich wie S3), `postgres`, das `Flyte Dashboard` und einen `contour` ingress controller installiert.

Installiert `flyte-core`

In [ ]:
!helm install flyte flyteorg/flyte-core -n flyte -f https://raw.githubusercontent.com/flyteorg/flyte/master/charts/flyte-core/values-sandbox.yaml --wait

Überzeugt euch, dass die Pods da sind

In [ ]:
!kubectl get pods -n flyte

`-n flyte` steht für `--namespace flyte`.

Öffnet nun unter http://127.0.0.1:30081/console/ das Flyte Dashboard im Browser.

**NOTE** Wir hatten den k3d Cluster so konfiguriert, das manche Ports falls möglich automatisch auf localhost verfügbar gemacht werden. Falls der Port 30081 nicht verfügbar ist müssen wir diesen manuell in einem anderen Terminalfenster verfügbar machen: 

```bash
kubectl port-forward svc/flyte-deps-contour-envoy 30081:80 -n flyte
```

Dieser Befehlt leitet localhost:30081 auf Port 80 des Service `flyte-deps-contour-envoy` weiter, welcher sich um die Weiterleitung zum Flyte Dashboard kümmert.

### 3 Pipeline packagen, registrieren und ausführen

#### 3.1 Docker image bauen und pushen

Unser Flyte Workflow benötigt ein Image in dem u.A. alle Dependencies enthalten sind, welche in den Flyte Workflow importiert werden. Wir bauen dieses Image mit dem Dockerfile `./Dockerfile`

In [ ]:
!docker build -t localhost:5000/workflow:latest .

In [ ]:
!docker push localhost:5000/workflow:latest

Schaut euch an was in einem Container mit diesem Image enthalten ist:

In [ ]:
!docker run -it localhost:5000/workflow ls

Der eigentliche Workflow code ist im Docker Image noch nicht enthalten. Flyte erstellt bei der Registrierung des Workflows ein tar Archiv, welches in einem Bucket gespeichert wird. Die Worker, die die tasks des Workflows ausführen, laden den Code aus diesem Bucket heruntern. Dies erlaubt schnelles Iterieren bei der Entwicklung des Workflows, da das Docker Image nur erneut gebaut werden muss, wenn neue Requirements hinzugefügt werden.

#### 3.2 Packen und registrieren des Workflows  (`workflow_part1.py`)
Dieser befindet sich in `flytesnacks/workflows/workflow_part1.py` und wird durch das folgende Script registriert:

In [ ]:
!pyflyte --config flyte_config register --project flytesnacks\
        --image k3d-registry.localhost:5000/workflow:latest\
        --version 1 flytesnacks/workflows/workflow_part1.py

#### 3.3 Manuelles lunchen des Workflows

Nun öffnet den registrierten Workflow im Flyte Dashboard, [hier](http://localhost:30081/console/projects/flytesnacks/domains/development/workflows/flytesnacks.workflows.workflow_part1.pipeline), und klickt auf `Lunch Workflow`.

Vergewissert euch, dass die Tasks erfolgreich durchgelaufen sind

In [ ]:
!kubectl get pods -n flytesnacks-development

Zeigt euch die Logs für den ersten Tasks `n0` an

In [ ]:
!kubectl get pods -n flytesnacks-development | grep 'n0' | awk '{print $1}' | xargs -L 1 kubectl -n flytesnacks-development logs 

Zeigt euch die Logs für den zweiten Tasks `n1` an

In [ ]:
!kubectl get pods -n flytesnacks-development | grep 'n1' | awk '{print $1}' | xargs -L 1 kubectl -n flytesnacks-development logs 

Zeigt euch die Logs für den dritten Tasks `n2` an

In [ ]:
!kubectl get pods -n flytesnacks-development | grep 'n2' | awk '{print $1}' | xargs -L 1 kubectl -n flytesnacks-development logs 

Logs von einem Kubernetes Pod zeigt man grundsätzlich folgendermaßen an:

```console
kubectl get pods -n <namespace name>   # Finde pod id heraus
kubectl -n <namespace name> logs -f <pod id>
```

### 4 Automatische Workflow Registrierung

#### 4.1 Git Repository initialisieren

In [ ]:
%%bash
if git rev-parse --git-dir > /dev/null 2>&1; then
  : # This is a valid git repository (but the current working
    # directory may not be the top level.
    # Check the output of the git rev-parse command if you care)
    echo "Warning, this notebook assumes one starts from a fresh git repository!"
else
  : 
  git init
  git add .
  git branch -m main
  git commit -m "First commit"
fi

0. Erstellt ein **privates** github repository
1. Substituiert euren GitHub Username für `<YOUR_GITHUB_USER>` unten
2. 
    - Erzeugt einen persönlichen Access Token, hier: https://github.com/settings/tokens/new
    - Gebt die Permissions für Repo und klickt `Generate token`
        - [x] repo
          - [x] repo:status
          - [x] repo_deployment
          - [x] public_repo
          - [x] repo:invite
          - [x] security_events
    - Substituiert euren GitHub Token für `<YOUR_GITHUB_TOKEN>` unten
3. Substituiert eure GitHub Repo unten.
4. Führt die nächste Zelle aus und **entfernt sie anschließend, damit das Token nicht in der Output Zelle des notebooks gespeichert wird!**



**YOUR GITHUB TOKEN IS LIKE A PASSWORD TO YOUR GITHUB ACCOUNT, DO NOT SHARE!**


In [ ]:
%%bash
cat > .env <<EOF
GITHUB_USER=<YOUR_GITHUB_USER>
GITHUB_TOKEN=<YOUR_GITHUB_TOKEN>
GITHUB_REPO=https://github.com/<owner>/<repo>.git
GITHUB_REPO_SSH=git@github.com:<owner>/<repo>.git
EOF

Ladet die Variablen der Environment

In [ ]:
import dotenv
dotenv.load_dotenv()

Pusht den Code auf den main branch euer remote Repository

In [ ]:
!git remote add origin ${GITHUB_REPO_SSH}
!git push -u origin main


#### 4.2 Installiert Argo CD

In [ ]:
!kubectl create ns argo

Installiert Argo CD in den Cluster

In [ ]:
!kubectl apply -n argo -f https://raw.githubusercontent.com/argoproj/argo-workflows/master/manifests/quick-start-postgres.yaml

Wartet bis die Pods gestartet sind

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pods --all -n argo

Überzeugt euch, dass die Pods da sind

In [ ]:
!kubectl get pods -n argo

Macht den Port 2746 lokal verfügbar, indem ihr in einem neuen Terminalfenster den folgenden Befehl ausführt: 
```bash
kubectl -n argo port-forward svc/argo-server 2746:2746
```
Anschließend könnt ihr das `Argo CD Dashboard` auf https://localhost:2746 öffnen

Erzeugt das Kubernetes Secret in `argo_cred.yaml`, damit der Argo Workflow im Cluster auf die GitHub Repository zugreifen kann. Wir substituieren die Environment Variablen mit envsubst:

In [ ]:
!echo $GITHUB_REPO

In [ ]:
!cat argo_cred.yaml | envsubst | kubectl apply -f -

#### 4.3 Erstellt den Argo Workflow (`workflow_part1.py`)
Der folgende Argo Workflow könnte von einem CI/CD Workflow gestartet werden, sodass bei jedem Commit in `flytesnacks/workflows` der Flyte Workflow gestartet wird. So wäre der Stand des Codes mit den Experimenten verknüpft.

Der folgende Befehlt sollte eure GitHub Repo zeigen (HTTPS Link):

In [ ]:
!echo $GITHUB_REPO

Erzeugt den Argo Workflow in `argo_cicd_workflow.yaml`. Indem ihr die Environment Variable `WORKFLOW=workflow_part1` mit envsubst substituiert wird der Flyte Workflow `flytesnacks/workflows/workflow_part1.py` gepackt, registriert und ausgeführt.

**Note** `Argo Workflow` (CD) triggers `Flyte Workflow` (MlOps)

In [ ]:
%env WORKFLOW=workflow_part1
!cat argo_cicd_workflow.yaml | envsubst | kubectl -n argo create -f -

Den Status des CICD workflows kann man unter [https://localhost:2746/workflows?limit=500](https://localhost:2746/workflows?limit=500) sehen.

1. Zunächst wird das Github repository geclont.
2. Dann wird der Flyte workflow registriert und ausgeführt.

Den Status des Flyte workflows kann man in der  [Flyte Console](http://127.0.0.1:30081/console/projects/flytesnacks/domains/development/workflows/flytesnacks.workflows.workflow_part1.pipeline) sehen.

Schaut, dass der Argo Workflow erfolgreich durchläuft:

In [ ]:
!kubectl get workflow -n argo

Schaut, dass der Workflow in Flyte durchläuft, es sollten nacheinander die Pods mit der Endung `-n0-0`, `-n1-0` und `-n2-0` auftauchen:

In [ ]:
!kubectl get pods -n flytesnacks-development

# Session 2

In [ ]:
import dotenv
dotenv.load_dotenv()

## 5 Mlflow-Tracking-Server Deployment

#### 5.1 Baut das Docker Image für Mlflow

In [ ]:
!docker build -t localhost:5000/mlflow:latest infrastructure/mlflow_tracking_server

In [ ]:
!docker push localhost:5000/mlflow:latest

Deployt Mlflow im Cluster. Dafür wird das Mlflow Image benötigt

In [ ]:
!kubectl apply -f infrastructure/mlflow_tracking_server/manifests/namespace.yaml
!kubectl apply -f infrastructure/mlflow_tracking_server/manifests

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pods --all -n mlflow

Überzeugt euch, dass die Pods da sind

In [ ]:
!kubectl get pods -n mlflow

#### 5.2 Minimales Mlflow Beispiel

**Wichtig:** Macht Mlflow auf dem lokalen Port 5001 verfügbar, indem ihr in einem **neuen Terminalfenster** den **folgenden Befehl ausführt**: 
```bash
kubectl -n mlflow port-forward svc/mlflow-server-service 5001:5000
```
Anschließend öffnet das `Mlflow Dashboard` auf http://127.0.0.1:5001

Führt dieses minimale Mlflow Experiment aus und schaut, dass im `Mlflow Dashboard` ein neuer Run mit Metrik `foo` auftaucht.

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.start_run()
mlflow.log_metric("foo", 1)
mlflow.end_run()

#### 5.3 Mlflow in der automatisierten Pipeline (`workflow_part2.py`)

Erzeugt den Argo Workflow in `argo_cicd_workflow.yaml`. Indem ihr die Environment Variable `WORKFLOW=workflow_part2` mit envsubst substituiert wird der Flyte Workflow `flytesnacks/workflows/workflow_part2.py` gepackt, registriert und ausgeführt.

**Note** `Argo Workflow` (CD) triggered `Flyte Workflow` (MlOps)

In [ ]:
%env WORKFLOW=workflow_part2
!cat argo_cicd_workflow.yaml | envsubst | kubectl -n argo create -f -

Schaut, dass der Argo Workflow erfolgreich durchläuft:

In [ ]:
!kubectl get workflow -n argo

Schaut, dass der Workflow in Flyte durchläuft, es sollten nacheinander die Pods mit der Endung `-n0-0`, `-n1-0` und `-n2-0` auftauchen:

In [ ]:
!kubectl get pods -n flytesnacks-development

Öffnet das [Mlflow Dashboard](http://127.0.0.1:5001) und schaut, dass ein neuer Run angelegt, entsprechende Parameter und Metriken gelogged und Artifakte registriert worden sind.

# Session 3

In [ ]:
import dotenv
dotenv.load_dotenv()

### 6 Deployment eines Models mit `Mlflow serve`

#### 6.1 (optional) Datensichtung im MinIO Bucket
Hierzu müsst ihr leider nochmals den MinIO Client (mc) installieren. Z.B. mit `brew install minio/stable/mc` oder siehe die Installationsseite, [hier](https://min.io/docs/minio/linux/reference/minio-mc.html#quickstart).

Schaut euch die Daten in unserem minio Bucket aus dem Flyte Deployment an.

In [ ]:
!mc alias set flyte http://localhost:30084 minio miniostorage --api S3v4

In [ ]:
!mc ls flyte/my-s3-bucket/breast_cancer_test_size_0.3/

In [ ]:
!mc head -n 2 flyte/my-s3-bucket/breast_cancer_test_size_0.3/X_test.csv

#### 6.2 Deployen eines Modells mit `Mlflow serve`

Führt den folgenden Befehl in einem **seperaten Terminalfenster** mit eurer **aktivierten venv** aus: 
```bash
mlflow models serve -m workshop_material/model -p 5002 \
    --env-manager=local
```

#### 6.3 Inference des mit `Mlflow serve` deployten Modells

In [ ]:
%%bash
curl --silent http://127.0.0.1:5002/invocations -H 'Content-Type: application/json; format=pandas-records' \
-d '[{"mean radius": 13.0, "mean texture": 25.13, "mean perimeter": 82.61, "mean area": 520.2, "mean smoothness": 0.08369, "mean compactness": 0.05073, "mean concavity": 0.01206, "mean concave points": 0.01762, "mean symmetry": 0.1667, "mean fractal dimension": 0.05449, "radius error": 0.2621, "texture error": 1.232, "perimeter error": 1.657, "area error": 21.19, "smoothness error": 0.006054, "compactness error": 0.008974, "concavity error": 0.005681, "concave points error": 0.006336, "symmetry error": 0.01215, "fractal dimension error": 0.001514, "worst radius": 14.34, "worst texture": 31.88, "worst perimeter": 91.06, "worst area": 628.5, "worst smoothness": 0.1218, "worst compactness": 0.1093, "worst concavity": 0.04462, "worst concave points": 0.05921, "worst symmetry": 0.2306, "worst fractal dimension": 0.06291}]'

### 7 Manuelles Deployment eines Models mit `Seldon-Core`

#### 7.1 Installiert Istio

Seldon-Core kümmert sich zusammen mit Istio um das Traffic Management in unserem Cluster. Das erlaubt uns, **verschiedene Modelle auf ähnlichen Domainpfaden** zu erreichen, z.B.:
1. localhost:8080/seldon/inference/**breast-cancer-clf**/api/v1.0/predictions

2. localhost:8080/seldon/inference/**my-other-model**/api/v1.0/predictions



In [ ]:
!kubectl create namespace istio-system

In [ ]:
!kubectl apply -f infrastructure/service_mesh/istio_crds.yaml

In [ ]:
!kubectl apply -f infrastructure/service_mesh/

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pods --all -n istio-system

In [ ]:
!kubectl get pods -n istio-system

#### 7.2 Installiert Seldon-Core

Man kann sich Seldon-Core als einen Flask Wrapper vorstellen, der ein Model initialisiert und bei einem Inference-Request die `predict()` Methode ausführt.
```python
class Model:
    def __init__(self, ...):
    """Custom logic that prepares model.

    - Reusable servers: your_loader downloads model from
    remote repository.
    - Non-Reusable servers: your_loader loads model from 
    a file embedded in the image.
    """
    self._model = your_loader(...)

    def predict(self, features, names=[], meta=[]):
    """Custom inference logic.""""
    return self._model.predict(...)
```

In [ ]:
!kubectl create namespace inference 
!kubectl create namespace inference-test

In [ ]:
!helm install seldon-core seldon-core-operator \
    --repo https://storage.googleapis.com/seldon-charts \
    --set usageMetrics.enabled=true \
    --set istio.enabled=true \
    --namespace seldon-system --create-namespace

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pods --all -n seldon-system

In [ ]:
!kubectl get pods -n seldon-system

#### 7.3 Bauen des Classifier Docker Images

Wir containerisieren nun ein einfaches Klassifizierungsmodel:
```
tree flytesnacks/seldon/templates
flytesnacks/seldon/templates
├── Classifier.py
├── Dockerfile
└── seldon_deployment.yaml
```
In Classifier.py haben wir unsere Modellklasse

In [ ]:
!cat flytesnacks/seldon/templates/Classifier.py | grep -A 2 "def " 

In [ ]:
!docker build -t localhost:5000/classifier:latest flytesnacks/seldon/templates

In [ ]:
!docker push localhost:5000/classifier:latest

#### 7.4 Manuelles deployen des Classifiers

Kopiert die Modell-ID aus Mlflow und tragt sie im File `flytesnacks/seldon/templates/seldon_deployment.yaml` hier ein:
```yaml
21    value: "my-s3-bucket/0/<model-id>/artifacts/model/model.pkl"
```
Wenn ihr den MinIO Client installiert habt, könnt ihr die Modells-ID auch hier auslesen:

In [ ]:
!mc ls flyte/my-s3-bucket/0/

Tauscht die Modell-ID, wie zwei Zelle weiter oben beschrieben und wendet das Manifest an:

In [ ]:
!kubectl apply -f flytesnacks/seldon/templates/seldon_deployment.yaml

Schaut euch die erzeuten Resourcen an:

In [ ]:
!kubectl get seldondeployment -n inference

In [ ]:
!kubectl get deployments -n inference

In [ ]:
!kubectl get pods -n inference

In [ ]:
!kubectl get pods -n inference | grep breast-cancer | awk '{print $1}' | xargs kubectl describe pods -n inference | grep -A 50 Events

Führt in einem **neuen Terminalfenster** diesen Befehl aus:
```bash
kubectl -n istio-system port-forward svc/istio-ingressgateway 8080:80
```
Dies macht das Istio Ingressgateway auf Port 8080 verfügbar und somit können wir alle deployten Modelle bequem erreichen.

In [ ]:
!curl http://localhost:8080/seldon/inference/breast-cancer-clf/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{"data": {"names": ["mean radius", "mean texture", "mean perimeter", "mean area", "mean smoothness", "mean compactness", "mean concavity", "mean concave points", "mean symmetry", "mean fractal dimension", "radius error", "texture error", "perimeter error", "area error", "smoothness error", "compactness error", "concavity error", "concave points error", "symmetry error", "fractal dimension error", "worst radius", "worst texture", "worst perimeter", "worst area", "worst smoothness", "worst compactness", "worst concavity", "worst concave points", "worst symmetry", "worst fractal dimension"], "ndarray": [[13.0, 25.13, 82.61, 520.2, 0.084, 0.051, 0.012, 0.018, 0.167, 0.054, 0.262, 1.232, 1.657, 21.19, 0.006, 0.009, 0.006, 0.006, 0.012, 0.002, 14.34, 31.88, 91.06, 628.5, 0.122, 0.109, 0.045, 0.059, 0.231, 0.063]]}}'

In [ ]:
!kubectl delete -f flytesnacks/seldon/templates/seldon_deployment.yaml

### 8 Automatisches Deployment eines Classifiers mit `Seldon-Core`

In [ ]:
import dotenv
dotenv.load_dotenv()

Erstellt einen Service Account, welcher Seldondeployments im Namespace flyte erstellen darf. 

Wended dazu das rbac.yaml File an (Role-based Access Controll).

In [ ]:
!kubectl create namespace seldon-logs
!kubectl apply -f infrastructure/flyte/rbac.yaml

Zudem brauchen wir noch ein Secret mit dem Github Access Token im namespace flyte, um Pull Request in unserer Repository zu stellen.

In [ ]:
!cat infrastructure/flyte/flyte_cred.yaml | envsubst | kubectl apply -f -

Schaut euch den erstellten Service Account an.

In [ ]:
!kubectl get ServiceAccount -n flytesnacks-development

#### 8.1 Installieren von ArgoCD aus dem ArgoProject

Wir nutzen ArgoCD als GitOps Tool, welches alle Manifeste im Ordner `infrastructure/inference` automatisch anwendet.

In [ ]:
!kubectl create namespace argocd

In [ ]:
!kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s pods --all -n argocd

In [ ]:
!kubectl get pods -n argocd

Auch im Namespace argocd brauchen wir ein Secret mit den Github Access Token, damit wir die Repository vom Cluster aus pullen können.

In [ ]:
!cat infrastructure/argocd/argocd_cred.yaml | envsubst | kubectl apply -f -

In [ ]:
!echo $GITHUB_REPO

In [ ]:
!cat infrastructure/argocd_application_inference.yaml | envsubst | kubectl apply -f -

In [ ]:
!kubectl get app -n argocd

In [ ]:
!kubectl describe app inference -n argocd | grep Events -A 20

#### 8.2 Starten des CI Workflows und automatisches Deployment

Vergewissert euch, dass die Env Variable GITHUB_REPO richtig gesetzt ist.

In [ ]:
!echo $GITHUB_REPO

Der `deploy_model()` Task des Flyte Workflows erstellt einen PR in eurer Repository. Dieser PR enthält ein Kubernetes Manifest für ein Seldondeployment eures Modells, welches im Ordner `infrastructure/inference` gespeichert wird. Sobald ihr den PR merged wird das **Modell automatisch deployed** (das Manifest wird durch unsere ArgoCD GitOps Application auf dem Cluster angewandt).

In [ ]:
%env WORKFLOW=workflow_part3
!cat argo_cicd_workflow.yaml | envsubst | kubectl -n argo create -f -

Die CI Pipeline registriert den Flyte Workflow und führt diesen im Namespace `flytesnacks-development` aus.

Schaut euch den inference-test Namespace an, hier wird das Modell kurzzeitig deployed und getested.

In [ ]:
!kubectl get pods -n inference-test

In [ ]:
!kubectl get seldondeployment -n inference-test

#### 8.3 Bestätigen des automatischen Deployments mit `Seldon-Core`

Geht auf eure GitHub Repository:

In [ ]:
!echo $GITHUB_REPO

Hier wurde ein Pull Request erstellt, welchen ihr zum bestätigen des Deployments mergen müsst. Dieser PR enthält das `Manifest eines SeldonDeployments`, welches in `infrastructure/inference` hinzugefügt wird.

Schaut euch den inference Namespace an, hier wird das Modell deployt.

In [ ]:
!kubectl get seldondeployment -n inference
!kubectl get pods -n inference

#### 8.4 Inference auf dem automatisch deployten Modell

Macht den folgenden port-forward in einem **neuen Terminalfenster**:
```bash
kubectl -n istio-system port-forward svc/istio-ingressgateway 8080:80
```

In [ ]:
!curl http://localhost:8080/seldon/inference/sklearn-random-forest-2/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{"data": {"names": ["mean radius", "mean texture", "mean perimeter", "mean area", "mean smoothness", "mean compactness", "mean concavity", "mean concave points", "mean symmetry", "mean fractal dimension", "radius error", "texture error", "perimeter error", "area error", "smoothness error", "compactness error", "concavity error", "concave points error", "symmetry error", "fractal dimension error", "worst radius", "worst texture", "worst perimeter", "worst area", "worst smoothness", "worst compactness", "worst concavity", "worst concave points", "worst symmetry", "worst fractal dimension"], "ndarray": [[13.0, 25.13, 82.61, 520.2, 0.084, 0.051, 0.012, 0.018, 0.167, 0.054, 0.262, 1.232, 1.657, 21.19, 0.006, 0.009, 0.006, 0.006, 0.012, 0.002, 14.34, 31.88, 91.06, 628.5, 0.122, 0.109, 0.045, 0.059, 0.231, 0.063]]}}'